# Notebook to get public and personnal private data from Sorare API

In [ ]:
import requests
import json
import pandas as pd
import multiprocessing

In [ ]:
# write your request here.

query = """
{
    auctions {
        nodes {
            id
            bidsCount
            name
            creditCardFee
            minNextBid
        }
    }
}
"""

In [ ]:
# Test request on Sorare API.

url = 'https://api.sorare.com/graphql/'
r = requests.post(url, json={'query': query})
print('request status:', r.status_code)
data = r.json()
r.json()

In [ ]:
print(len(data['data']['allCards']['nodes']))
for element in data['data']['allCards']['nodes']:
    print(element)

In [ ]:
# script to fetch basics data about cards and owners and save it in .xlsx
# Please adjust hyperparameters in case processing is too long.

import time

has_next_page = True
count = 1

df = pd.DataFrame()
card_name = []
card_slug = []
card_created_at = []
card_rarity = []
card_position = []
card_club = []
card_price = []
user_name = []
user_created_at = []
user_slug = []

count_no_common = 0

while has_next_page:
    print(count)
    if count == 1:
        query = """
        {
          allCards(first: 50) {
            pageInfo {
              hasNextPage
              hasPreviousPage
              startCursor
              endCursor
            }
            nodes {
              name
              slug
              createdAt
              rarity
              position
              club {
                  name
              }
              price
              user {
                nickname
                slug
                createdAt
              }
            }
          }
        }
        """
    else:
        query = '''
        {
          allCards(first: 50 after: "''' + last_cursor_ + '''") {
            pageInfo {
              hasNextPage
              hasPreviousPage
              startCursor
              endCursor
            }
            nodes {
              name
              slug
              createdAt
              rarity
              position
              club {
                  name
              }
              price
              user {
                nickname
                slug
                createdAt
              }
            }
          }
        }
        '''

    url = 'https://api.sorare.com/graphql/'
    r = requests.post(url, json={'query': query})
    time.sleep(0)
    data = r.json()
    print(data['data']['allCards']['pageInfo']['hasPreviousPage'])
    print(data['data']['allCards']['pageInfo']['hasNextPage'])
    last_cursor_ = data['data']['allCards']['pageInfo']['endCursor']
    has_next_page = data['data']['allCards']['pageInfo']['hasNextPage']
    print(last_cursor_)

    for element in data['data']['allCards']['nodes']:
        if element['rarity'] != 'common':
            count_no_common = count_no_common + 1
            card_name.append(element['name'])
            card_created_at.append(element['createdAt'])
            card_rarity.append(element['rarity'])
            card_slug.append(element['slug'])
            card_price.append(element['price'])
            card_position.append(element['position'])
            card_club.append(element['club']['name'])
            try:
                user_name.append(element['user']['nickname'])
                user_created_at.append(element['user']['createdAt'])
                user_slug.append(element['user']['slug'])
            except:
                user_name.append(None)
                user_created_at.append(None)
                user_slug.append(None)
    count = count + 1
    print('count_no_common: ', count_no_common)

df['card'] = card_name
df['card_slug'] = card_slug
df['card_created_at'] = card_created_at
df['card_rarity'] = card_rarity
df['card_position'] = card_position
df['card_club'] = card_club
df['card_price_eth'] = list(map(lambda x : x/pow(10,18), (map(float, card_price))))
df['user_created_at'] = user_created_at
df['user_slug'] = user_slug

df.to_excel('data_sorare.xlsx')

In [ ]:
df

In [ ]:
# Possible optimization using multiprocessing task (map/reduce operations)...

# Get personnal account data

## Prerequisites :
- Only working when 2FA is deactivated.
- Please wait another upgrade to take into account 2FA.

## Fill your personnal data in the cell below :

In [ ]:
my_email = # type your email here
my_password = # type your password here

In [ ]:
import requests
import json

url = 'https://api.sorare.com'
extension = '/api/v1/users/' + my_email
path = url + extension
r = requests.get(path)
data = r.json()

my_salt = data['salt']
my_token = r.headers['CSRF-TOKEN']
my_cookie = r.headers['Set-Cookie']
my_sorare_session = my_cookie.split('_sorare_session_id=')[1].split(';')[0]

print('my_salt:', my_salt)
print('\n')
print('my_token:', my_token)
print('\n')
print('my_cookie:', my_cookie)
print('\n')
print('my_sorare_session:', my_sorare_session)

In [ ]:
import bcrypt

if not isinstance(my_salt, bytes):
    my_password = my_password.encode('utf-8')
if not isinstance(my_salt, bytes):
    my_salt = my_salt.encode('utf-8')

# Hash a password for the first time, with a randomly-generated salt
hashed = bcrypt.hashpw(my_password, my_salt)
# Check that an unhashed password matches one that has previously been
# hashed
if bcrypt.checkpw(my_password, hashed):
    print("It Matches!")
else:
    print("It Does not Match :(")

In [ ]:
url = 'https://api.sorare.com'
extension = '/users/sign_in.json'
path = url + extension
my_payload = {
    "user": {
        "email": my_email,
        "password": hashed.decode('utf-8')
    }
}
my_headers = {
    'Content-Type': 'application/json; charset=utf-8',
    "X-CSRF-TOKEN": my_token
}
my_cookie = {
    '_sorare_session': my_sorare_session,
}
r = requests.post(path, data = json.dumps(my_payload), headers = my_headers, cookies = my_cookie)

In [ ]:
my_new_cookie = r.headers['Set-Cookie']
my_new_token = r.headers['CSRF-TOKEN']
my_etag = r.headers['ETag']
my_new_sorare_session = my_new_cookie.split('_sorare_session_id=')[1].split(';')[0]

In [ ]:
r.json()

In [ ]:
query = """
{
  currentUser {
    availableBalance
    availableBalanceForWithdrawal
    bankMappedEthereumAddress
    bankMappedEthereumAddressApprovedBank
    betaTester
    blocked {
      slug
    }
    blockedBy {
      slug
    }
    blockedUntil
    buyingAuctions {
      name
      number
      open
      slug
      bidsCount
    }
    cardCounts {
      common
      rare
      superRare
      total
      unique
    }
    cardsCount
    createdAt
    depositedEth
    directOffersSent {
      aasmState
    }
    directOffersReceived {
      aasmState
    }
    draftAvailableFunds
    email
    ethereumAddress
    fiatEnabled
    firstName
    fromPath
    frontChatUserHash
    lastName
    nickname
    onboardingStatus {
      enabled
      skipped
    }
    otpProvisioningUri
    otpRequiredForLogin
    phoneNumber
    phoneNumberVerificationRequested
    profile {
      clubName
      discordUsername
      rookie
      slug
      status
    }
    referee
    referrals {
      pages
      totalCount
      referrals {
        refereeCardsBoughtAtAuctionCount
        refereeConfirmedAt
        refereeIdentification
        refereeInvitationSentAt
      }
    }
    referrer {
      slug
    }
    slug
    sorareAddress
    sorareAddressApprovedBank
    sorareAddressApprovedBankPending
    sorarePrivateKey {
      encryptedPrivateKey
      iv
      salt
    }
    sorarePrivateKeyRecovery
  }
}
"""

my_headers = {
    'Content-Type': 'application/json; charset=utf-8',
    "X-CSRF-TOKEN": my_new_token,
    'ETag': my_etag
}
my_cookie = {
    '_sorare_session': my_new_sorare_session
}

url = 'https://api.sorare.com/graphql/'
r = requests.post(url, json={'query': query}, headers = my_headers, cookies = my_cookie)
print(r.status_code)
data = r.json()
r.json()